In [1]:
import operator
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = '../res/Proper'
MODELS_DIR = '/Models'
IMG_SIZE_X = 224
IMG_SIZE_Y = 224
LR = 1e-3
labels = os.listdir(TRAIN_DIR)
LABELS_COUNT = len(labels)
MODEL_NAME = 'resnet-cactusifier-{}-{}-{}-12conv2filter2size-02.model'.format(LR, '2conv-basic','treshold_60')

labelCounts = {}
for directory in labels:
    files = os.listdir(os.path.join(TRAIN_DIR, directory))
    labelCounts[directory] = len(files)

print(sum(labelCounts.values()), 'images')
labelCounts = sorted(labelCounts.items(), key=operator.itemgetter(1), reverse = True)
labelCounts

1427 images


[('E Horizontalonius', 178),
 ('A Retusus', 175),
 ('A Myriostigma', 163),
 ('A Kotschoubeyanus', 156),
 ('Th Bicolor', 40),
 ('L Williamsii', 28),
 ('S Disciformis', 27),
 ('R Beguinii', 26),
 ('A valdezii', 24),
 ('C humilis', 24),
 ('H curvispinus', 23),
 ('N Conoidea', 23),
 ('Th Heterochromus', 23),
 ('A Trigonus', 22),
 ('H kunzei', 22),
 ('N subgibbosa', 22),
 ('Ef ochoterenaus', 21),
 ('H heinrichianus', 20),
 ('Lobivia pampana', 20),
 ('Th esmeraldana', 20),
 ('A lloydii', 19),
 ('C Echinus', 19),
 ('Ep Greggii', 19),
 ('H limariensis', 19),
 ('H Texensis', 19),
 ('T nikolae', 18),
 ('Th Multicephalus', 18),
 ('Thelocephala challensis', 18),
 ('A ritteri', 17),
 ('M Candida', 17),
 ('N senilis', 17),
 ('T Valdezianus', 17),
 ('A Fissuratus', 16),
 ('Th Hex', 16),
 ('Th lloydii', 16),
 ('A Coahuilense', 15),
 ('A scaphar', 15),
 ('Ec Pectinatus', 15),
 ('H taltalensis', 15),
 ('M lasiacantha', 15),
 ('Th napina', 15),
 ('Thelocephala glabrescens', 15)]

In [2]:
def get_hot_one_encoded_label(label):
    h1e = np.zeros(LABELS_COUNT)
    h1e[labels.index(label)] = 1
    return h1e

In [3]:
def get_label_from_h1e(h1e):
    index = np.where(h1e == 1)[0][0]
    return labels[index]

In [4]:
def get_label_from_index(index):
    return labels[index]

In [5]:
def create_train_data():
    training_data = []
    counter = 0
    for directory in labels:
        print(counter, directory)
        labelDirectory = os.path.join(TRAIN_DIR, directory)
        files = os.listdir(labelDirectory)
        for img in files:
            label = get_hot_one_encoded_label(directory)
            path = os.path.join(labelDirectory, img)
            try:
                img = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (IMG_SIZE_X, IMG_SIZE_Y))
            except Exception as e:
                print(path)
                print(str(e))
            training_data.append([np.array(img), np.array(label)])
            #training_data.append([np.divide(np.array(img), 255), np.array(label)])
        counter = counter + 1
    print('Shuffling')
    shuffle(training_data)
    print('Saving')
    #np.save('train_data.npy', training_data)
    print('Done!')
    return training_data

In [6]:
train_data = create_train_data()

0 A Kotschoubeyanus
1 A Myriostigma
2 A Retusus
3 E Horizontalonius
Shuffling
Saving
Done!


In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None, IMG_SIZE_X, IMG_SIZE_Y, 3], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, LABELS_COUNT, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


In [ ]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print("Model", MODEL_NAME, "loaded!")

In [ ]:
trainCountRatio = 0.8
trainSize = int(round(len(train_data)*trainCountRatio))
test = train_data[:-trainSize]
train = train_data[-trainSize:]

In [ ]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE_X, IMG_SIZE_Y, 3)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE_X, IMG_SIZE_Y, 3)
test_y = [i[1] for i in test]

In [ ]:
for i in range(300):
    model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}), 
        snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
    model.save(MODEL_NAME)

##################################

In [ ]:
model.save(MODEL_NAME)

In [ ]:
TEST_PATH = '../res/Test'
TEST_SAMPLE_PATH = os.path.join(TEST_PATH, 'myriostigma.jpg')
testImage = np.array(cv2.resize(cv2.imread(TEST_SAMPLE_PATH, cv2.IMREAD_COLOR), (IMG_SIZE_X, IMG_SIZE_Y)))
#testImage = np.divide(np.array(cv2.resize(cv2.imread(TEST_SAMPLE_PATH, cv2.IMREAD_COLOR), (IMG_SIZE_X, IMG_SIZE_Y))), 255)
testImage = testImage.reshape(-1, IMG_SIZE_X, IMG_SIZE_Y, 3)
result = model.predict(testImage)
s = sorted(enumerate(result[0]), key=lambda x: x[1])
bestMatches = s[-5:]
for match in bestMatches:
    print(get_label_from_index(match[0]), str(round(match[1]*100, 2)) + '%')

In [ ]:
#tensorboard --logdir=foo:E:\Programming\Python\Cactusifier\app\log